### 네이버 검색 결과 크롤링
- 영등포역 맛집
- Selenium 이용

In [28]:
import requests
import pandas as pd
from bs4 import BeautifulSoup
from urllib.parse import quote

- requests.get()으로 데이터를 가져오는 경우

In [29]:
base_url = 'https://search.naver.com/search.naver?where=view&sm=tab_jum'
url = f'{base_url}&query={quote("영등포역 맛집")}'
res =requests.get(url)
soup = BeautifulSoup(res.text)

In [30]:
lis = soup.select('.bx._svp_item')
len(lis)

30

- Selenium 이용해서 데이터를 가져오는 경우

In [31]:
from selenium import webdriver

driver = webdriver.Chrome()
driver.get(url)

In [32]:
# View 페이지를 아래로 내려서 많은 데이터 확보
from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys
import time

body = driver.find_element(By.TAG_NAME, 'body')
for _ in range(30):
    body.send_keys(Keys.PAGE_DOWN)
    time.sleep(1)

In [33]:
soup = BeautifulSoup(driver.page_source)
lis = soup.select('.bx._svp_item')
len(lis)

110

In [34]:
driver.close()

- 날짜, 제목, 내용 크롤링

In [35]:
li = lis[0]
title = li.select_one('.api_txt_lines.total_tit._cross_trigger').get_text(strip=True)
title

'영등포역 맛집영등갈비 미묘하게 매력적인 고기집'

In [36]:
content = li.select_one('.api_txt_lines.dsc_txt').get_text(strip=True)
content[:200]

'최근에영등포역 맛집영등갈비 방문 후 포스팅 해봅니다 난생 처음 먹어본 색다른 갈비 였네요 영등포역... 서울 영등포구 영신로 19번길 1 영업시간: 오후 12시 - 밤 10시 브레이크타임: 오후 2시 - 오후 4시 (평일) 연락처: 0507...'

- 시간

In [37]:
date = li.select_one('.sub_time.sub_txt').get_text().strip()
date

'2023.08.28.'

In [42]:
from datetime import datetime, timedelta

now = datetime.now()
now

datetime.datetime(2023, 9, 18, 15, 18, 22, 452847)

In [45]:
# 어제
date = now - timedelta(1)
f'{date.year}.{date.month:02d}.{date.day:02d}'

'2023.09.17'

In [46]:
# 2일 전
date = now - timedelta(2)
date.strftime('%Y.%m.%d')

'2023.09.16'

In [47]:
# 23시간 전
date = now - timedelta(hours=23)
date.strftime('%Y.%m.%d')

'2023.09.17'

In [49]:
# 40분 전
date = now - timedelta(minutes=40)
date.strftime('%Y.%m.%d')

'2023.09.18'

In [50]:
def convert_date(s):
    if len(s) == 11:    # '2023.09.18'
        return s[:-1]

    now = datetime.now()
    if '어제' in s:
        date = now - timedelta(1)
    elif '일' in s:
        date = now - timedelta(int(s[0]))
    elif '시간' in s: 
        date = now - timedelta(hours=int(s[:-4]))
    elif '분' in s:
        date = now - timedelta(minutes=int(s[:-3]))
    
    return date.strftime('%Y.%m.%d')

- Step 4

In [51]:
data = []
for li in lis:
    date = convert_date(li.select_one('.sub_time.sub_txt').get_text().strip())
    title = li.select_one('.api_txt_lines.total_tit._cross_trigger').get_text(strip=True)
    content = li.select_one('.api_txt_lines.dsc_txt').get_text(strip=True)
    data.append({'날짜': date, '제목': title, '내용': content})

In [52]:
df = pd.DataFrame(data)
df.tail()

,날짜,제목,내용
105,2023.09.10,[음식점리뷰]영등포역 맛집교촌치킨에서 맛있는 치킨 먹어볼까요?,그래서 퇴근하자마자 집 근처에 있는영등포역 맛집교촌치킨으로 직행했다. 저번에 테이크...
106,2023.09.06,영등포역인근맛집추천해주세요!!!!,영등포역인근맛집추천해주세요!!!!
107,2023.09.18,영등포역맛집이고집쭈꾸미 여전히맛있어 최고야,이고집쭈꾸미 본점 #영등포역맛집#영등포맛집 #이고집쭈꾸미 #영등포쭈꾸미 친구들이랑 ...
108,2023.04.03,꽃구경 코스로 좋은영등포역 맛집광부맥주,꽃구경 코스로 좋은영등포역 맛집광부맥주 주말에 집에있다가 날씨가 너무 좋아서 당장 ...
109,2023.04.05,영등포역한식식당배달전문 음식점 주방담당 및 보조 직원 & 포장 알바직원 구인합니다.,영등포역5분거리에 있는 한식배달식당에서 직원 및 포장 직원 구인합니다. ■ 주5일 ...
